In [1]:
import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install tensorflow
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

'c:\Program' is not recognized as an internal or external command,
operable program or batch file.


In [2]:

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [4]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):] 


In [5]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')


Epoch 1/200
21/21 [==============================] - 2s 5ms/step - loss: 2.8611 - accuracy: 0.0971
Epoch 2/200
21/21 [==============================] - 0s 4ms/step - loss: 2.7482 - accuracy: 0.0777
Epoch 3/200
21/21 [==============================] - 0s 4ms/step - loss: 2.5838 - accuracy: 0.1456
Epoch 4/200
21/21 [==============================] - 0s 5ms/step - loss: 2.4598 - accuracy: 0.2330
Epoch 5/200
21/21 [==============================] - 0s 5ms/step - loss: 2.3616 - accuracy: 0.2524
Epoch 6/200
21/21 [==============================] - 0s 4ms/step - loss: 2.2384 - accuracy: 0.3204
Epoch 7/200
21/21 [==============================] - 0s 4ms/step - loss: 2.1087 - accuracy: 0.3495
Epoch 8/200
21/21 [==============================] - 0s 5ms/step - loss: 1.9755 - accuracy: 0.3883
Epoch 9/200
21/21 [==============================] - 0s 5ms/step - loss: 1.7874 - accuracy: 0.4854
Epoch 10/200
21/21 [==============================] - 0s 4ms/step - loss: 1.7211 - accuracy: 0.4757
Epoch 11/